# Protein Data Bank (PDB)
# 蛋白质数据获取：蛋白质数据银行（PDB）


### 内容在理论中 ¶

-  蛋白质数据银行（PDB）
- 查询 PDB 使用 Python 包 `biotite` 和 `pypdb`

###  内容在实用篇中

-  选择一个查询蛋白
- 获取查询蛋白的 PDB 条目数量
- 查找满足特定条件的 PDB 条目
- 选择分辨率最高的 PDB 条目
- 获取顶级结构中配体的元数据
- 绘制顶配体分子
- 创建蛋白质-配体 ID 对
- 对齐 PDB 结构并提取配体

###  参考文献

- 蛋白质数据银行（PDB 网站）
- `pypdb` Python 软件包（生物信息学（2016），1，159-60；文档）
- `biotite` Python 软件包（BMC 生物信息学（2018），19；文档）
- 分子叠加与 Python 包 `opencadd` （仓库）

##  理论 ¶

### 蛋白质数据银行（PDB）

RCSB 蛋白质数据银行（PDB）是一个全面的生物信息学数据库，是结构生物学领域，如结构基因组学和药物设计等关键资源（PDB 网站）。

结构数据来源于结构确定方法，如 X 射线晶体学（最常见方法）、核磁共振（NMR）和冷冻电子显微镜（cryo-EM）。对于每个条目，数据库包含（i）蛋白质、配体、辅因子、水分子和离子的原子 3D 坐标以及连接这些原子的键，以及（ii）关于结构数据的元信息，如 PDB ID、作者、沉积日期、使用的结构确定方法和结构分辨率。结构分辨率是收集数据质量的度量，单位为Å（埃）；值越低，结构质量越高。

### 查询 PDB 使用 Python 包 `biotite` 和 `pypdb` ¶

query one ( `FieldQuery` ，见文档) 或更多 ( `CompositeQuery` ，见文档) 的这些字段以检索符合我们标准的 PDB ID 的数量 ( `count` ) 或列表 ( `search` )。

In [48]:
import collections
import logging
import pathlib
import time
import warnings
import datetime

import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
from tqdm.auto import tqdm
import redo
import requests_cache
import nglview
import pypdb
import biotite.database.rcsb as rcsb
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools

from opencadd.structure.superposition.api import align, METHODS
from opencadd.structure.core import Structure

# Disable some unneeded warnings
logger = logging.getLogger("opencadd")
logger.setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

# Cache requests -- this will speed up repeated queries to PDB
requests_cache.install_cache("rcsb_pdb", backend="memory")

In [49]:
from pathlib import Path
import os

# 获取当前工作目录
HERE = Path(os.getcwd())
DATA = HERE / 'data_P31751'
if not DATA.exists():
    DATA.mkdir(parents=True, exist_ok=True)
print(DATA)


/Users/wangyang/Desktop/Liver_cancer/16-AIDD/06_Protein_data_acquisition-Protein_Data_Bank_(PDB)/data_P31751


### 选择查询蛋白 ¶

我们使用 AURKA 作为本演讲的查询蛋白。 AURKA 的 UniProt ID 为 `O14965` ，下面将使用该 ID 来查询 PDB 数据库。

In [50]:
uniprot_id = "P31751"

### 获取查询蛋白质的 PDB 条目数 ¶

AURKA 的 PDB 中有多少可用结构（在本笔记本上次运行时）？

In [51]:
query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
today = datetime.datetime.now()
print(
    f"结构数 {today.year}-{today.month}-{today.day}: {rcsb.count(query_by_uniprot_id)}"
)

结构数 2024-11-23: 19


自 1971 年 PDB 成立以来，每年有多少个可用结构？

我们定义我们的标准。

In [52]:
uniprot_id = "P31751"
before_deposition_date = "2024-11-15T00:00:00Z"
experimental_method = "X-RAY DIFFRACTION"
max_resolution = 3.0
n_chains = 1
min_ligand_molecular_weight = 100.0

我们设置每个查询。

In [53]:
query_by_uniprot_id = rcsb.FieldQuery(
    "rcsb_polymer_entity_container_identifiers.reference_sequence_identifiers.database_accession",
    exact_match=uniprot_id,
)
query_by_deposition_date = rcsb.FieldQuery(
    "rcsb_accession_info.deposit_date", less=before_deposition_date
)
query_by_experimental_method = rcsb.FieldQuery("exptl.method", exact_match=experimental_method)
query_by_resolution = rcsb.FieldQuery(
    "rcsb_entry_info.resolution_combined", less_or_equal=max_resolution
)
query_by_polymer_count = rcsb.FieldQuery(
    "rcsb_entry_info.deposited_polymer_entity_instance_count", equals=n_chains
)
query_by_ligand_mw = rcsb.FieldQuery(
    "chem_comp.formula_weight", molecular_definition=True, greater=min_ligand_molecular_weight
)

我们单独执行每个查询并检查每个条件的匹配数。

In [54]:
print(f"具有Uniprot蛋白数据库身份标识号的结构数量 {uniprot_id}: {rcsb.count(query_by_uniprot_id)}")
time.sleep(0.1)  # wait shortly to not overload the API
print(
    f"之前沉积的结构数量 {before_deposition_date}: {rcsb.count(query_by_deposition_date)}"
)
time.sleep(0.1)
print(
    f"分解的结构数 {experimental_method}: {rcsb.count(query_by_experimental_method)}"
)
time.sleep(0.1)
print(
    f"分辨率小于或等于的结构数 {max_resolution}: {rcsb.count(query_by_resolution)}"
)
time.sleep(0.1)
print(f"仅有的结构数量 {n_chains} chain: {rcsb.count(query_by_polymer_count)}")
time.sleep(0.1)
print(
    f"配体大于或等于的结构数目 {min_ligand_molecular_weight} Da: {rcsb.count(query_by_ligand_mw)}"
)

具有Uniprot蛋白数据库身份标识号的结构数量 P31751: 19
之前沉积的结构数量 2024-11-15T00:00:00Z: 227561
分解的结构数 X-RAY DIFFRACTION: 189357
分辨率小于或等于的结构数 3.0: 183870
仅有的结构数量 1 chain: 80316
配体大于或等于的结构数目 100.0 Da: 227560


我们将所有查询与 and 运算符结合起来，以仅匹配满足所有条件的 PDB ID。

In [55]:
query = rcsb.CompositeQuery(
    [
        query_by_uniprot_id,
        query_by_deposition_date,
        query_by_experimental_method,
        query_by_resolution,
        query_by_polymer_count,
        query_by_ligand_mw,
    ],
    "and",
)
pdb_ids = rcsb.search(query)
print(f"匹配数量s: {len(pdb_ids)}")
print("选择了Pdb IDS:")
print(*pdb_ids)

匹配数量s: 7
选择了Pdb IDS:
1GZK 1GZN 1GZO 1MRV 1MRY 8Q61 9C1W


### 选择最高分辨率的 PDB 条目 ¶

到目前为止，我们已经使用某些搜索条件来查找感兴趣的 PDB 条目。

目前，我们无法直接通过 `biotite` 访问结构体的分辨率；使用 `biotite` 我们只能检查分辨率是否满足特定条件。相反，我们下载所选 PDB ID 的完整元数据。为此，我们使用 `pypdb` 包中的方法 `describe_pdb` 。每个结构的元数据都作为字典返回。

注意：我们在这里只获取 PDB 结构的元信息，我们还没有获取结构（3D 坐标）。

redo.retriable 行是一个装饰器。这包装了该函数并提供了额外的功能。在这种情况下，它将自动重试失败的查询（最多 10 次）。

In [56]:
@redo.retriable(attempts=10, sleeptime=2)
def describe_one_pdb_id(pdb_id):
    """Fetch meta information from PDB."""
    described = pypdb.describe_pdb(pdb_id)
    if described is None:
        print(f"! Error while fetching {pdb_id}, retrying ...")
        raise ValueError(f"Could not fetch PDB id {pdb_id}")
    return described

In [57]:
pdbs_data = [describe_one_pdb_id(pdb_id) for pdb_id in tqdm(pdb_ids)]

  0%|          | 0/7 [00:00<?, ?it/s]

让我们看一下第一个 PDB ID 的元数据（键只是因为字典包含很多信息，我们不想在这里打印）。在 PDB 结构和 PDBx/mmCIF 格式初学者指南中查找有关 PDB 元数据的更多信息。

In [58]:
print("\n".join(pdbs_data[0].keys()))

audit_author
cell
citation
diffrn
diffrn_detector
diffrn_radiation
diffrn_source
entry
exptl
exptl_crystal
exptl_crystal_grow
pdbx_audit_revision_category
pdbx_audit_revision_details
pdbx_audit_revision_group
pdbx_audit_revision_history
pdbx_audit_revision_item
pdbx_database_related
pdbx_database_status
pdbx_initial_refinement_model
rcsb_accession_info
rcsb_entry_container_identifiers
rcsb_entry_info
rcsb_primary_citation
refine
refine_analyze
refine_hist
refine_ls_restr
reflns
software
struct
struct_keywords
symmetry
rcsb_id


让我们仔细看看我们感兴趣的两个键：包含 PDB ID ( "id" ) 的 "entry" 键和包含 PDB ID ( "id" ) 的 "rcsb_entry_info" 键。其中包含结构的分辨率 ( "resolution_combined" )。

In [59]:
pdbs_data[0]["entry"]

{'id': '1GZK'}

In [60]:
pdbs_data[0]["rcsb_entry_info"]

{'assembly_count': 1,
 'branched_entity_count': 0,
 'cis_peptide_count': 0,
 'deposited_atom_count': 2481,
 'deposited_deuterated_water_count': 0,
 'deposited_hydrogen_atom_count': 0,
 'deposited_model_count': 1,
 'deposited_modeled_polymer_monomer_count': 271,
 'deposited_nonpolymer_entity_instance_count': 0,
 'deposited_polymer_entity_instance_count': 1,
 'deposited_polymer_monomer_count': 315,
 'deposited_solvent_atom_count': 284,
 'deposited_unmodeled_polymer_monomer_count': 44,
 'diffrn_radiation_wavelength_maximum': 0.9792,
 'diffrn_radiation_wavelength_minimum': 0.9792,
 'disulfide_bond_count': 0,
 'entity_count': 2,
 'experimental_method': 'X-ray',
 'experimental_method_count': 1,
 'inter_mol_covalent_bond_count': 0,
 'inter_mol_metalic_bond_count': 0,
 'molecular_weight': 36.49,
 'na_polymer_entity_types': 'Other',
 'nonpolymer_entity_count': 0,
 'polymer_composition': 'homomeric protein',
 'polymer_entity_count': 1,
 'polymer_entity_count_dna': 0,
 'polymer_entity_count_rna':

现在，我们将每个 PDB ID 的分辨率保存为 pandas DataFrame，按分辨率升序排序。

In [61]:
resolution = pd.DataFrame(
    [
        [pdb_data["entry"]["id"], pdb_data["rcsb_entry_info"]["resolution_combined"][0]]
        for pdb_data in pdbs_data
    ],
    columns=["pdb_id", "resolution"],
).sort_values(by="resolution", ignore_index=True)
resolution

,pdb_id,resolution
0,9C1W,2.00
1,1GZK,2.30
2,8Q61,2.32
3,1GZN,2.50
4,1GZO,2.75
5,1MRV,2.80
6,1MRY,2.80


从顶层结构获取配体的元数据 ¶

In [62]:
top_num = 10 # 顶部结构数量

In [63]:
top_num

10

在下一节中，我们将以最高分辨率从顶部 top_num 结构构建基于配体的整体药效团

In [64]:
selected_pdb_ids = resolution[:top_num]["pdb_id"].to_list()
print(f"选定的 PDB ID: {selected_pdb_ids}")

选定的 PDB ID: ['9C1W', '1GZK', '8Q61', '1GZN', '1GZO', '1MRV', '1MRY']


选定的最高分辨率 PDB 条目可以包含针对不同结合位点的配体，例如变构和正构配体，这会阻碍基于配体的药效团的生成。因此，我们将重点关注以下 4 个结构，它们在正位结合口袋中含有配体。笔记本后面提供的代码可用于验证这一点。

In [65]:
# 获取pdb_ids前面10个
pdb_ids = pdb_ids[:10]
pdb_ids

['1GZK', '1GZN', '1GZO', '1MRV', '1MRY', '8Q61', '9C1W']

In [66]:
selected_pdb_ids = pdb_ids  
print(f"Selected PDB IDs (frozen set): {selected_pdb_ids}")

Selected PDB IDs (frozen set): ['1GZK', '1GZN', '1GZO', '1MRV', '1MRY', '8Q61', '9C1W']


们使用 get_ligands 获取有关顶级 top_num 配体的 PDB 信息，并将其存储为 csv 文件（作为每个配体的字典）。

如果一个结构包含多个配体，我们选择最大的配体。注意：这是在蛋白质结合位点选择配体的简单但不全面的方法。该方法还可以选择与蛋白质结合的辅因子。因此，在进一步使用之前，请目视检查自动选择的顶级配体。

In [67]:
import os
import collections
import requests
import pandas as pd
from Bio.PDB import PDBParser
from rdkit import Chem
from rdkit.Chem import Draw, PandasTools
import mdtraj as md
import nglview

# 定义下载和解析PDB文件的函数
def get_ligands_from_pdb_file(pdb_id):
    pdb_filename = DATA / f"{pdb_id}.pdb"
    
    # 检查本地是否已有文件，若无则下载
    if not os.path.exists(pdb_filename):
        pdb_file_url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
        response = requests.get(pdb_file_url)
        
        if response.status_code != 200:
            print(f"Failed to download PDB file for ID {pdb_id}")
            return {}

        with open(pdb_filename, "wb") as file:
            file.write(response.content)
    
    # 使用BioPython解析PDB文件
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure(pdb_id, pdb_filename)

    ligands = {}
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.id[0] != " " and residue.resname != "HOH":  # 跳过水分子
                    ligand_id = residue.resname
                    atoms = list(residue.get_atoms())
                    molecular_weight = sum(atom.mass for atom in atoms if atom.mass)

                    # 使用RCSB PDB API获取配体的SMILES
                    smiles = fetch_smiles_from_rcsb(ligand_id)
                    
                    ligands[ligand_id] = {
                        "Name": ligand_id,
                        "Molecular weight": molecular_weight,
                        "Formula": "N/A",
                        "InChI descriptor": "N/A",
                        "InChIKey descriptor": "N/A",
                        "Stereo SMILES (OpenEye)": smiles if smiles else "N/A",
                    }

    return ligands

# 使用RCSB PDB API获取SMILES的辅助函数
def fetch_smiles_from_rcsb(compound_id):
    url = f"https://data.rcsb.org/rest/v1/core/chemcomp/{compound_id}"
    response = requests.get(url)
    if response.status_code == 200:
        try:
            smiles = response.json()["rcsb_chem_comp_descriptor"]["smiles"]
            return smiles
        except KeyError:
            return None
    else:
        print(f"SMILES not found for {compound_id} in RCSB PDB.")
        return None

# 示例获取和解析配体信息
rows = []
for pdb_id in selected_pdb_ids:
    ligands = get_ligands_from_pdb_file(pdb_id)
    if ligands:
        for ligand_id, properties in ligands.items():
            rows.append([
                pdb_id,
                ligand_id,
                properties["Name"],
                properties["Molecular weight"],
                properties["Formula"],
                properties["InChI descriptor"],
                properties["InChIKey descriptor"],
                properties["Stereo SMILES (OpenEye)"]
            ])
    else:
        print(f"No ligands found for {pdb_id}")

# 将数据转换为DataFrame并保存为CSV文件
columns = [
    "@structureId",
    "@chemicalID",
    "chemicalName",
    "@molecularWeight",
    "formula",
    "InChI",
    "InChIKey",
    "smiles"
]
ligands_df = pd.DataFrame(rows, columns=columns)
ligands_df.to_csv(DATA / "PDB_top_ligands.csv", header=True, index=False)




No ligands found for 1GZK
No ligands found for 1GZN
No ligands found for 1GZO
No ligands found for 1MRV
No ligands found for 1MRY


In [68]:
# 读取ligands_df，如果@chemicalID为数字，则删除这一整行
ligands_df = pd.read_csv(DATA / "PDB_top_ligands.csv")
ligands_df = ligands_df[ligands_df["@chemicalID"].apply(lambda x: not x.isdigit())]
ligands_df.head()


,@structureId,@chemicalID,chemicalName,@molecularWeight,formula,InChI,InChIKey,smiles
0,8Q61,K06,K06,414.307200,NaN,NaN,NaN,CCCN1c2cc(c(nc2OCC1=O)c3ccc(cc3)C4(CC(C4)(C)O)...
1,8Q61,GOL,GOL,84.030300,NaN,NaN,NaN,C(C(CO)O)O
2,8Q61,PO4,PO4,94.971361,NaN,NaN,NaN,[O-]P(=O)([O-])[O-]
3,8Q61,MES,MES,182.133500,NaN,NaN,NaN,C1COCC[NH+]1CCS(=O)(=O)[O-]
4,9C1W,XOO,XOO,540.614420,NaN,NaN,NaN,c1ccc(cc1)c2ccc3c(n2)n(c(n3)c4cccnc4N)c5ccc(cc...


In [69]:
# 保存选定的PDB ID
ligands_df.to_csv(DATA / "PDB_top_ligands.csv", header=True, index=False)




In [70]:
# 添加分子列并绘制顶部配体分子
PandasTools.AddMoleculeColumnToFrame(ligands_df, "smiles")
top_num = 4
Draw.MolsToGridImage(
    mols=list(ligands_df.ROMol.dropna()),  # 跳过无效的分子
    legends=list(ligands_df["@chemicalID"] + ", " + ligands_df["@structureId"]),
    molsPerRow=top_num
)

# 创建蛋白质-配体 ID 对
pairs = collections.OrderedDict(zip(ligands_df["@structureId"], ligands_df["@chemicalID"]))
print(pairs)

# 使用MDTraj加载和对齐PDB结构，并选择主链原子（N, CA, C）
structures = []
reference_structure = None
excluded_pdb_ids = []

for i, pdb_id in enumerate(pairs):
    traj = md.load(DATA / f"{pdb_id}.pdb")
    main_chain_atoms = traj.topology.select("name N or name CA or name C")  # 选择主链原子
    main_chain_traj = traj.atom_slice(main_chain_atoms)
    
    # 设置第一个匹配的结构为参考结构
    if reference_structure is None:
        reference_structure = main_chain_traj
        structures.append(main_chain_traj)
    else:
        # 检查与参考结构的原子数量是否一致
        if main_chain_traj.n_atoms == reference_structure.n_atoms:
            structures.append(main_chain_traj)
        else:
            excluded_pdb_ids.append(pdb_id)
            print(f"Excluded PDB ID {pdb_id} due to mismatched atom count")

aligned_structures = []
for structure in structures:
    aligned = structure.superpose(reference_structure)
    aligned_structures.append(aligned)

print("Excluded PDB IDs due to atom count mismatch:", excluded_pdb_ids)

# 使用nglview可视化对齐后的结构
view = nglview.NGLWidget()
for aligned in aligned_structures:
    view.add_component(aligned)
view.render_image(trim=True, factor=2, transparent=True)

# 提取配体并保存
ligands = []
for pdb_id in pairs.keys():
    if pdb_id not in excluded_pdb_ids:  # 跳过被排除的结构
        structure = md.load(DATA / f"{pdb_id}.pdb")
        # 提取配体部分 (假设配体为非标准残基)
        ligand_atoms = structure.topology.select("not protein and resname != 'HOH'")
        if len(ligand_atoms) == 0:
            print(f"No ligand atoms found for PDB ID {pdb_id}, skipping.")
            continue
        ligand = structure.atom_slice(ligand_atoms)
        ligand.save_pdb(DATA / f"{pdb_id}_lig.pdb")
        ligands.append(ligand)

# 检查配体文件
ligand_files = [file for file in os.listdir(DATA) if file.endswith("_lig.pdb")]
print(ligand_files)



OrderedDict([('8Q61', 'MES'), ('9C1W', 'EDO')])
Excluded PDB ID 9C1W due to mismatched atom count
Excluded PDB IDs due to atom count mismatch: ['9C1W']
['8Q61_lig.pdb']


In [71]:
# 使用nglview可视化提取的配体
view = nglview.NGLWidget()
for ligand in ligands:
    view.add_component(ligand)
    view.add_licorice()
view.render_image(trim=True, factor=2, transparent=True)
view._display_image()